# CONSTRUCCIÓN DE UN ASISTENTE CHATBOT  PQRS

Fecha: 30/11/2024
### Integrantes

- Jose Manuel Villa Romero
- Sebastian Vasquez Grajales

consiste en construir un sistema de transformacion de frases y cadenas textuales en conjuntos de palabras faciles de interpretar por el modelo de machine learning haciendo uso de tecnicas gramaticales para limpiar, simplificar, dividir, clasificar y reducir terminos para luego usarlos como patrones de aprendizaje. el enfoque que se busca con este ejercicio es aplicar el proceso de tratamiento y transformacion a un conjunto de frases provenientes de un set de datos de reseñas, comentarios y opiniones, y con base a ello generar una bolsa de palabras a partir de las palabras que componen dichas frases. el flujo de trabajo es el siguiente:

- **1.** Generacion del algoritmo de tratamiento de datos y representacion para:
     - Tokenización
     - Limpieza
     - Eliminación de stopwords
     - Lematización
     - Stemming (si aplica)
     - Bag of words
- **2.** Clasificación del conjunto de etiquetas, y construcción del dataframe con:
     -  Saludo
     -  Queja
     -  Reclamo
     -  Petición
     -  Despedida
     -  Información de contacto
     -  Horario de atención
     -  Precios
     -  Política de devolución
     -  Soporte técnico
- **3.** Diseño de modelo de machine learning
     - Entrenamiento de Naive Bayes extendido(clasificacion multiclase)
     - Evaluacion y ajuste de rendimiento
     - Validacion final
       
## 1. Generación del algoritmo de tratamiento de datos y representación 

In [2]:
import pandas as pd
import spacy
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer


diccionario = {
    "Saludo": ["Hola", "Buenos días", "Buenas tardes", "¿Cómo estás?", "Hola, ¿cómo están?", 
               "Muy buenos días, ¿pueden ayudarme?", "Buenas tardes, necesito información.", 
               "Hola, ¿hay alguien disponible?", "¡Saludos!", "¿Qué tal?", "¡Qué tal todo!", 
               "¡Hola, espero que estén bien!", "¡Buenos días! ¿Qué tal todo?", "¡Buenas tardes! ¿Cómo puedo ayudarte?", 
               "Hola, ¿qué tal todo por ahí?"],
    "Queja": ["Quisiera hacer una queja", "No estoy satisfecho con el servicio", "El producto llegó dañado", 
              "No me ha gustado la atención recibida", "El servicio fue muy lento", "La calidad del producto no es buena",
              "Estoy insatisfecho con lo que recibí", "El envío fue demorado", "No estoy contento con la compra", 
              "El producto no cumplió con mis expectativas", "El servicio no fue como esperaba", "Me siento decepcionado con la compra"],
    "Reclamo": ["Necesito reclamar por un error", "Quiero hacer un reclamo", "Esto es inaceptable, quiero una solución",
                "Estoy muy molesto, necesito hacer un reclamo.", "Quiero reportar un error con mi pedido.",
                "Esto no cumple con lo que prometieron, quiero una solución.", "El producto no funciona, ¿cómo hago un reclamo?",
                "Quiero mi dinero de vuelta, el producto llegó en mal estado.", "Esto es un error grave, necesito ayuda.",
                "No me han respondido a tiempo, quiero hacer un reclamo", "Este problema es urgente, necesito que lo resuelvan ya", 
                "Mi producto no funciona, quiero saber qué hacer"],
    "Petición": ["Me gustaría pedir información", "Necesito saber más detalles sobre el producto", "Por favor, envíenme más información", 
                 "¿Pueden darme más detalles?", "Quisiera saber más sobre sus servicios", "Por favor, envíenme una cotización", 
                 "Me gustaría conocer las características del producto", "¿Podrían enviarme más información por correo?", 
                 "Estoy interesado en saber más", "¿Tienen algún catálogo de productos?", "¿Me pueden dar detalles adicionales?", 
                 "¿Cómo puedo obtener más información sobre lo que ofrecen?"],
    "Despedida": ["Adiós", "Hasta luego", "Nos vemos", "chao", "hasta pronto", "gracias", "Cuídate", "Hasta la próxima", 
                  "Que tengas un buen día", "Nos vemos pronto", "Hasta entonces", "Gracias por todo", "Me voy, adiós", 
                  "Hasta pronto, que estés bien", "Cuidate, nos vemos"],
    "Información de contacto": ["¿Cómo puedo contactarlos?", "¿Cuál es el número de teléfono?", "¿Tienen un número de WhatsApp?", 
                               "¿Cuál es su correo electrónico?", "¿Cómo me comunico con el soporte?", "¿Tienen atención telefónica?", 
                               "¿Dónde los puedo ubicar?", "¿Puedo contactarlos por chat?", "¿Tienen un formulario de contacto?", 
                               "¿Dónde puedo encontrar información de contacto?", "¿Puedo llamarlos para soporte?", "¿Tienen línea directa?"],
    "Horario de atención": ["¿Cuál es el horario de atención?", "¿En qué días están abiertos?", "¿A qué hora abren?", 
                            "¿Cuáles son los horarios para atención al cliente?", "¿Tienen horarios especiales?", 
                            "¿Puedo visitar la tienda en fin de semana?", "¿Abren los domingos?", "¿Cuál es el horario de apertura?", 
                            "¿Hasta qué hora están abiertos?", "¿Qué días atienden?", "¿El horario es el mismo todos los días?", 
                            "¿Puedo hacer una llamada fuera del horario habitual?"],
    "Precios": ["¿Cuánto cuesta el producto?", "¿Cuál es el precio de servicio?", "¿Cuánto cuesta este producto?", 
                "¿Podrían decirme el precio del servicio?", "¿Hay descuentos disponibles?", "¿Es este el precio final o tiene impuestos adicionales?", 
                "Necesito saber los costos antes de decidirme.", "¿El precio incluye envío?", "¿Tienen precios para empresas?", 
                "¿Ofrecen planes de pago?", "¿Este precio es válido para todos los productos?", "¿El precio cambia según la cantidad?"],
    "Política de devolución": ["¿Cuál es la política de devoluciones?", "¿Puedo devolver el producto?", "¿Hasta cuándo puedo devolverlo?", 
                               "¿Cuáles son los requisitos para una devolución?", "¿Tengo que pagar por el envío de la devolución?", 
                               "¿Puedo cambiar un producto?", "¿Cómo proceso una devolución?", "¿Puedo devolverlo si está usado?", 
                               "¿Cuánto tiempo tarda el reembolso?", "¿Qué debo hacer si el producto llega defectuoso?", "¿Puedo devolverlo en la tienda?"],
    "Soporte técnico": ["Tengo un problema técnico", "¿Cómo puedo solucionar un error?", "Tengo un problema con el producto, ¿pueden ayudarme?", 
                        "¿Cómo resuelvo un error en el sistema?", "No sé cómo instalarlo, ¿me pueden dar soporte técnico?", "El dispositivo no enciende, ¿qué puedo hacer?", 
                        "Hay un fallo recurrente en mi aplicación, necesito ayuda.", "El sistema se cae, ¿cómo lo soluciono?", "¿Me pueden ayudar con la configuración?", 
                        "Mi dispositivo no funciona, ¿qué debo hacer?", "No puedo acceder a mi cuenta, ¿pueden ayudarme?", "¿Cómo restauro el sistema?"]
}


# Cargar el modelo de idioma español de spaCy
nlp = spacy.load("es_core_news_md")

#SPACY PARA TOKENIZAR, LIMPIAR, LEMATIZAR Y ELIMINAR STOPWORDS
def transformar_frase(frase):
    doc = nlp(frase)  
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])


data = []

#DEVUELVE UN ARREGLO PLANO CON CADA UNA DE LAS FRASES DENTRO DE LOS VALUES.    
for categoria, frases in diccionario.items():
    for frase in frases:
        texto_procesado = transformar_frase(frase)
        data.append([texto_procesado, categoria])

data = np.array(data)
# print(data)
df = pd.DataFrame(data, columns=["Texto transformado", "Clase"])
vacios= df[df['Texto transformado']=='']['Texto transformado'].index.tolist()
df.drop(vacios,axis=0,inplace=True)
df.reset_index(drop=True,inplace=True)
# CREAMOS EL VECTORIZADOR PARA LA BOLSA DE PALABRAS
vectorizer = CountVectorizer() 

# ALIMENTAR EL VECTORIZADOR Y CONVERTIR LA LISTA DE PALABRAS EN UNA MATRIZ DE FRECUENCIAS (BOLSA DE PALABRAS)
bagwords=vectorizer.fit_transform(df['Texto transformado']).toarray()

# CREAR EL DATAFRAME
df_bagword = pd.DataFrame(bagwords, columns=vectorizer.get_feature_names_out())

# SE AGREGA LA COLUMNA DE ETIQUETAS
df_bagword['Clase'] = df['Clase']

df_bagword.head()

,abierto,abrir,acceder,adicional,adiós,alguien,apertura,aplicación,atención,atender,...,urgente,usado,ver,visitar,vuelta,válido,whatsapp,yo,él,Clase
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Saludo
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Saludo
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Saludo
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Saludo
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Saludo


## 2. VERIFICACIÓN DE LAS ETIQUETAS
Debido a que en el paso 1 se efectuo paralelamente la union de las etiquetas con e conjunto de caracteristicas, este paso se precisa de una verificación de que todas las clases se hayan relacionado correctamente, inspeccionando el resumen de valores en la columna "Clase".

In [3]:
df_bagword['Clase'].value_counts()

Clase
Queja                      12
Reclamo                    12
Horario de atención        12
Petición                   12
Despedida                  12
Información de contacto    12
Soporte técnico            12
Precios                    12
Saludo                     11
Política de devolución     11
Name: count, dtype: int64

## 3. APLICACIÓN DEL MODELO DE MACHINE LEARNING (REGRESIÓN LOGÍSTICA) 
Se desea aplicar un modelo multiclase naturalmente categorico, que proporcionara las prestaciones adecuadas para llevar a cabo la estimación. Se configura a continuacion el modelo haciendo uso de una evaluacion basica de rendimiento.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


x= df_bagword.drop(['Clase'],axis=1)
y= df_bagword['Clase']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
print('test: ',y_test.unique(),'\n')
print('train: ',y_train.unique())



test:  ['Precios' 'Despedida' 'Información de contacto' 'Soporte técnico'
 'Horario de atención' 'Petición' 'Reclamo' 'Política de devolución'
 'Saludo' 'Queja'] 

train:  ['Información de contacto' 'Política de devolución' 'Petición' 'Precios'
 'Soporte técnico' 'Reclamo' 'Despedida' 'Saludo' 'Queja'
 'Horario de atención']


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Definir los parámetros a ajustar
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Parámetro de regularización
    'solver': ['liblinear', 'lbfgs'],  # Algoritmos de optimización
    'max_iter': [100, 500, 1000],  # Número máximo de iteraciones
    'penalty': ['l2', 'l1'],  # Tipo de regularización
}

# Crear el modelo base
model = LogisticRegression(random_state=1)

# Definir GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajustar el modelo con los mejores parámetros encontrados
grid_search.fit(x_train, y_train)

# Mejor combinación de parámetros
print("Mejores parámetros encontrados:", grid_search.best_params_)

# Predecir con el modelo ajustado
best_model = grid_search.best_estimator_
prediccion = best_model.predict(x_test)

# Métricas de evaluación

### VALIDACIÓN DE SUB Y SOBRE AJUSTE ###
y_train_pred = best_model.predict(x_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", train_accuracy)

y_test_pred = best_model.predict(x_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

### EVALUACIÓN ###
print("Classification Report:\n", classification_report(y_test, prediccion))
print("Confusion Matrix:\n", confusion_matrix(y_test, prediccion))


Mejores parámetros encontrados: {'C': 10, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Train Accuracy: 1.0
Test Accuracy: 0.6666666666666666
Classification Report:
                          precision    recall  f1-score   support

              Despedida       0.43      1.00      0.60         3
    Horario de atención       1.00      0.75      0.86         4
Información de contacto       1.00      0.67      0.80         3
               Petición       0.50      1.00      0.67         1
 Política de devolución       1.00      0.80      0.89         5
                Precios       1.00      1.00      1.00         2
                  Queja       0.00      0.00      0.00         1
                Reclamo       0.33      1.00      0.50         1
                 Saludo       0.00      0.00      0.00         1
        Soporte técnico       0.00      0.00      0.00         3

               accuracy                           0.67        24
              macro avg       0.53      0

C:\Users\sebas\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
90 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\sebas\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\sebas\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sebas\AppDa

# 4. CONSTRUIR EL PIPELINE DE CONSUMO

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
# Cargar el modelo de idioma español de spaCy
nlp = spacy.load("es_core_news_md")

#SPACY PARA TOKENIZAR, LIMPIAR, LEMATIZAR Y ELIMINAR STOPWORDS
def transformar_frase(frase):
    doc = nlp(frase)  
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])


def agregar_pregunta(pregunta):
    
    pregunta_procesada = transformar_frase(pregunta)
   
    pregunta_bagword = vectorizer.transform([pregunta_procesada]).toarray()
    prediccion = best_model.predict(pregunta_bagword)
    print(f"La clase predicha para la pregunta es: {prediccion[0]}")








In [ ]:
nueva_pregunta = input("Ingresa una nueva pregunta para predecir: ")
agregar_pregunta(nueva_pregunta)

El modelo de lenguaje natural no tiene un rendimiento maximo, pero sin embargo tiene una exactitud del 100% dentro del conjunto de datos de entrenamiento y un 60% de exactitud para datos nuevos. Para efectos practicos el rendimiento es suficiente ya que necesitara de mayor cantidad de datos interpretativos de cada clase para poder optimizar sus respuestas. La calidad y cantidad de datos es importante, sin embargo existen factores ajustables como el tipo de modelo, los hiperparametros y el preprocesamiento del conjunto de entrenamiento que deberan considerarse para maximizar las prestaciones del sistema de predicción.
